In [1]:
%load_ext autoreload
%autoreload 2

import helper_functions as hf
from imports import *
import importlib

num_available_cpus = multiprocessing.cpu_count()
print("Number of available CPUs:", num_available_cpus)

torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device =", device)
torch.set_default_tensor_type('torch.cuda.FloatTensor') if torch.cuda.is_available() else print ('cpu')

torch.set_num_threads(num_available_cpus)

print("Number of threads:", torch.get_num_threads())
print("Number of interop threads:", torch.get_num_interop_threads())

/nobackup/users/sambt/anaconda3/envs/quak/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of available CPUs: 80
Device = cuda:0
Number of threads: 80
Number of interop threads: 80


In [2]:
model = "XYY_X3000_Y80_UL17_clip10_NSRATQUAD_k6_hf120_nbpl4_tb10.pt"
bkg_model = "QCDBKG_clip10_NSRATQUAD_k6_hf120_nbpl4_tb10.pt"
flow = hf.load_model(name=model)
bkg_flow = hf.load_model(name=bkg_model)
bkg_data, bkg_unnorm, bkg_mass = hf.load_full("QCDBKG",num_batches=2)
bkg_mean = np.mean(bkg_unnorm,axis=0)
bkg_std = np.std(bkg_unnorm,axis=0)
sig, sig_unnorm, sig_mass = hf.load_full("XYY_X3000_Y80_UL17",bkg_mean=bkg_mean,bkg_std=bkg_std)

In [4]:
from scipy.stats import pearsonr

sig_losses = -flow.eval_log_prob(sig)[0]
bkg_losses = -flow.eval_log_prob(bkg_data)[0]

sig_losses_bkg = -bkg_flow.eval_log_prob(sig)[0]
bkg_losses_bkg = -bkg_flow.eval_log_prob(bkg_data)[0]

corr1, p1 = pearsonr(sig_mass[:,0],sig_losses)
corr2, p2 = pearsonr(sig_mass[:,0],sig_losses_bkg)
corr3, p3 = pearsonr(bkg_mass[:,0],bkg_losses)
corr4, p4 = pearsonr(bkg_mass[:,0],bkg_losses_bkg)

print("Sig mass - sigtr sig loss correlation: {0}".format(corr1))
print("Sig mass - bkgtr sig loss correlation: {0}".format(corr2))
print("Bkg mass - sigtr bkg loss correlation: {0}".format(corr3))
print("Bkg mass - bkgtr bkg loss correlation: {0}".format(corr4))

Sig mass - sigtr sig loss correlation: 0.11542115572656152
Sig mass - bkgtr sig loss correlation: 0.18844837481115487
Bkg mass - sigtr bkg loss correlation: 0.06319470052446327
Bkg mass - bkgtr bkg loss correlation: 0.12116013726804346


In [3]:
!nvidia-smi

Fri Jul  1 11:07:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   44C    P0    59W / 300W |    881MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
grid = np.arange(-10,10,step=0.1) + 0.05
num_pts = bkg_data.shape[0]
test_grid = np.tile(bkg_data,[len(grid),1])
test_grid[:,0] = np.repeat(grid,num_pts)
num_splits = test_grid.shape[0] // 100000
losses = []
t1 = time.time()
for subarr in np.array_split(test_grid,num_splits):
    loss = bkg_flow.eval_log_prob(subarr)[0]
    losses.append(loss)
t2 = time.time()
print("took {0:.4f} secs to process {1} events".format(t2-t1,test_grid.shape[0]))

took 108.9397 secs to process 42616200 events


In [12]:
#losses = np.concatenate(losses,axis=0)
loss_by_pt = []
for i in range(num_pts):
    loss_by_pt.append(0.1*np.sum(np.exp(losses[i::num_pts])))

In [14]:
print(bkg_data.shape)
print(len(loss_by_pt))

(213081, 14)
213081


In [21]:
bkg_unnorm[0]

array([1.5750000e+02, 3.5543817e-01, 8.4975237e-01, 8.4002590e-01,
       3.8215637e+00, 8.7158203e-02, 7.1369759e+01, 9.9609375e+00,
       7.3708010e-01, 7.6862401e-01, 7.9076397e-01, 1.8173405e+01,
       1.5954590e-01, 4.0570789e+01], dtype=float32)